In [12]:
import numpy as np
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import pyspark.sql.functions as fcn 
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.types import IntegerType, StringType, TimestampType

In [2]:
spark = SparkSession.builder.appName('UDF').getOrCreate()

In [6]:
from pyspark.sql.functions import col, pandas_udf
from pyspark.sql.types import LongType

# Declare the function and create the UDF
def multiply_func(a, b):
    return a * b

multiply = pandas_udf(multiply_func, returnType=LongType())

# The function for a pandas_udf should be able to execute with local Pandas data
x = pd.Series([1, 2, 3])
print(multiply_func(x, x))
# 0    1
# 1    4
# 2    9
# dtype: int64

# Create a Spark DataFrame, 'spark' is an existing SparkSession
df = spark.createDataFrame(pd.DataFrame(x, columns=["x"]))

# Execute function as a Spark vectorized UDF
df.select(multiply(col("x"), col("x"))).show()

0    1
1    4
2    9
dtype: int64
+-------------------+
|multiply_func(x, x)|
+-------------------+
|                  1|
|                  4|
|                  9|
+-------------------+



In [3]:
def MDY_to_YMD(s):
    
    '''
    The format to be converted is: '2012/5/1 0:00'
    '''
    
    split2 = s.split()
    sep_in = '/'
    sep_ou = '-'
    tokens = split2[0].split(sep_in)
    y = tokens[2]
    m = tokens[1]
    d = tokens[0]
    return y + sep_ou + m + sep_ou + d


'''
In your spark code use 
add.select(to_timestanp(udf(df.t)).alias(“dt”))
'''

MDY_to_YMD('1/5/2012 0:00')

'2012-5-1'

# 1 

In [20]:
def pd_StringToTime(s):    
    '''
    The current format to be converted is: '2012/5/1 0:00'
    '''
    return pd.to_datetime(s)

# 2

In [30]:
# Syntax: pyspark.sql.functions.pandas_udf(f=None, returnType=None, functionType=None)[source]
Time_udf = fcn.pandas_udf(pd_StringToTime, TimestampType() )   #StringType()

# 3

In [31]:
df = spark.createDataFrame([('1/5/2012 0:00',)], ['t'])
#df.select(fcn.to_timestamp(Time_udf(df.t)).alias('dt')).collect()
#df.select(Time_udf(df.t).alias('Date')).show()
#df.withColumn('Date',Time_udf(df.t)).show()
df.withColumn('Date',Time_udf(df.t)).drop('t').show()

+-------------------+
|               Date|
+-------------------+
|2012-01-05 00:00:00|
+-------------------+



# ==============================================================